In [10]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
from src.component.data_info import*

In [11]:
#read the data
demand_data=pd.read_parquet((TRANSFORMED_DATA_DIR / 'tabular_data.parquet'))
demand_data

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,target_demand_next_hour
0,11233.0,10916.0,10704.0,10387.0,10196.0,9905.0,9600.0,9254.0,9137.0,9045.0,...,9199.0,9215.0,8763.0,8954.0,8954.0,9607.0,2024-01-28 02:00:00,53,0.1085,10318.0
1,11360.0,11147.0,11114.0,10904.0,10317.0,9762.0,9408.0,9056.0,8839.0,8796.0,...,9245.0,8668.0,8401.0,8388.0,8419.0,8419.0,2024-01-29 01:00:00,53,0.1585,8861.0
2,12702.0,12591.0,12286.0,12122.0,11578.0,10916.0,10271.0,10111.0,9917.0,10061.0,...,10282.0,9847.0,8928.0,9189.0,9403.0,9696.0,2024-01-30 00:00:00,53,1.1585,9696.0
3,11807.0,12549.0,12565.0,12310.0,12062.0,11694.0,11073.0,10446.0,10039.0,10055.0,...,11168.0,10028.0,9399.0,9044.0,9114.0,8829.0,2024-01-31 00:00:00,53,6.3585,9193.0
4,10352.0,11299.0,12290.0,12391.0,12174.0,11998.0,11487.0,10970.0,10378.0,10148.0,...,11025.0,11324.0,11523.0,11523.0,11311.0,11422.0,2024-01-31 23:00:00,53,2.4085,11696.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29302,3221.0,3175.0,3154.0,3167.0,3199.0,3274.0,3372.0,3475.0,3522.0,3500.0,...,3354.0,3464.0,3464.0,3470.0,3425.0,3380.0,2024-12-28 04:00:00,71,-1.0415,3322.0
29303,3351.0,3294.0,3295.0,3338.0,3405.0,3495.0,3659.0,3955.0,4150.0,4147.0,...,3025.0,3080.0,3253.0,3253.0,3330.0,3333.0,2024-12-29 03:00:00,71,-2.3415,3307.0
29304,3866.0,3723.0,3641.0,3624.0,3635.0,3686.0,3778.0,3964.0,4255.0,4403.0,...,2998.0,2993.0,3047.0,3226.0,3226.0,3303.0,2024-12-30 02:00:00,71,2.2085,3325.0
29305,3950.0,3763.0,3589.0,3481.0,3407.0,3363.0,3329.0,3396.0,3561.0,3831.0,...,3551.0,3523.0,3482.0,3494.0,3616.0,3616.0,2024-12-31 01:00:00,71,1.6085,3629.0


In [12]:
#split the data
from datetime import datetime
from src.component.data_info import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    demand_data,
    cutoff_date=datetime(2024, 7, 1, 0, 0, 0),
    target_column_name='target_demand_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(13409, 675)
y_train.shape=(13409,)
X_test.shape=(15898, 675)
y_test.shape=(15898,)


In [13]:
# use only past demand data
# past_demand_columns = [c for c in X_train.columns if c.startswith('demand_')]
# X_train_only_numeric = X_train[past_demand_columns]
X_train_numurical=X_train.drop(['date'], axis=1)

## Baseline model (Linear regression)

In [14]:
#basleline model linear regrisson
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train_numurical, y_train)

LinearRegression()

In [15]:
X_test_numeric = X_test.drop(['date'], axis=1)
y_pred = model.predict(X_test_numeric)
y_pred

array([11604.4723634 , 12743.94414446, 12906.90694557, ...,
        3283.51697088,  3681.36916119,  3789.46332658], shape=(15898,))

In [16]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
def evaluate_model(y_test, y_pred):
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_pred)
    return f"MAE is {test_mae:.4f} and MAPE is: {test_mape:.4f}"
    

In [17]:
evaluate_model(y_test, y_pred)

'MAE is 77.2308 and MAPE is: 0.3007'

## XGboost model

In [18]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train_numurical, y_train)
print(model_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)


In [19]:

y_pred_XGB = model_xgb.predict(X_test_numeric)
y_pred_XGB

array([11859.244 , 12797.702 , 12718.079 , ...,  3367.3115,  3643.153 ,
        3733.514 ], shape=(15898,), dtype=float32)

In [20]:
evaluate_model(y_test, y_pred_XGB)

'MAE is 101.2659 and MAPE is: 0.2215'

## LightBM model

In [21]:
import lightgbm as lgb

In [22]:
model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train_numurical, y_train)
print(model_lgb)

d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171666
[LightGBM] [Info] Number of data points in the train set: 13409, number of used features: 674
[LightGBM] [Info] Start training from score 3558.845999
LGBMRegressor()


In [23]:
y_pred_lgb=model_lgb.predict(X_test_numeric)
y_pred_lgb

array([11534.61808081, 12988.42190566, 12792.46055584, ...,
        3234.42971373,  3563.76901432,  3754.18255275], shape=(15898,))

In [24]:
evaluate_model(y_test,y_pred_lgb)

'MAE is 102.4056 and MAPE is: 0.3918'

as we can see the lighbm perform better so we will choose it and do hyperprameter tunning

## lightbm with feature_engineering

In [25]:

def average_demand_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    """
    Adds one column with the average rides from
    - 7 days ago
    - 14 days ago
    - 21 days ago
    - 28 days ago
    """
    X['average_demand_last_4_weeks'] = 0.25*(
    X[f'demand_previous_{7*24}_hour'] + \
    X[f'demand_previous_{2*7*24}_hour'] + \
    X[f'demand_previous_{3*7*24}_hour'] + \
    X[f'demand_previous_{4*7*24}_hour']
    )
    return X

In [26]:
from sklearn.preprocessing import FunctionTransformer

add_feature_average_demand_last_4_weeks = FunctionTransformer(
    average_demand_last_4_weeks, validate=False)
     

In [27]:
add_feature_average_demand_last_4_weeks.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,average_demand_last_4_weeks
0,11233.0,10916.0,10704.0,10387.0,10196.0,9905.0,9600.0,9254.0,9137.0,9045.0,...,9199.0,9215.0,8763.0,8954.0,8954.0,9607.0,2024-01-28 02:00:00,53,0.108500,10073.50
1,11360.0,11147.0,11114.0,10904.0,10317.0,9762.0,9408.0,9056.0,8839.0,8796.0,...,9245.0,8668.0,8401.0,8388.0,8419.0,8419.0,2024-01-29 01:00:00,53,0.158500,10384.25
2,12702.0,12591.0,12286.0,12122.0,11578.0,10916.0,10271.0,10111.0,9917.0,10061.0,...,10282.0,9847.0,8928.0,9189.0,9403.0,9696.0,2024-01-30 00:00:00,53,1.158500,11313.25
3,11807.0,12549.0,12565.0,12310.0,12062.0,11694.0,11073.0,10446.0,10039.0,10055.0,...,11168.0,10028.0,9399.0,9044.0,9114.0,8829.0,2024-01-31 00:00:00,53,6.358500,11268.50
4,10352.0,11299.0,12290.0,12391.0,12174.0,11998.0,11487.0,10970.0,10378.0,10148.0,...,11025.0,11324.0,11523.0,11523.0,11311.0,11422.0,2024-01-31 23:00:00,53,2.408500,10960.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13404,4124.0,4124.0,4041.0,3928.0,3847.0,3624.0,3341.0,3116.0,2980.0,2893.0,...,4467.0,4751.0,5035.0,5315.0,5556.0,5698.0,2024-06-26 21:00:00,71,23.358500,3997.00
13405,3843.0,3763.0,3763.0,3660.0,3585.0,3506.0,3360.0,3162.0,2965.0,2848.0,...,4211.0,4503.0,4789.0,5033.0,5272.0,5476.0,2024-06-27 20:00:00,71,23.408501,4096.75
13406,4289.0,4395.0,4417.0,4417.0,4302.0,4122.0,3981.0,3758.0,3470.0,3263.0,...,3960.0,4143.0,4331.0,4522.0,4782.0,5080.0,2024-06-28 19:00:00,71,21.158501,4282.50
13407,4280.0,4421.0,4537.0,4517.0,4517.0,4399.0,4217.0,4060.0,3811.0,3491.0,...,3697.0,3826.0,4093.0,4365.0,4671.0,4921.0,2024-06-29 18:00:00,71,26.308500,4167.75


In [28]:

from sklearn.base import BaseEstimator, TransformerMixin
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
class TemporalFeaturesEngineer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        
        X_ = X.copy()
        X_['date']= pd.to_datetime(X_['date'], format='%Y-%m-%d %H:%M:%S')
        
        # Generate numeric columns from datetime
        X_["hour"] = X_['date'].dt.hour
        X_["day_of_week"] = X_['date'].dt.dayofweek
        X_['month'] = X_['date'].dt.month
        X_['is_weekend'] = X_['day_of_week'].isin([5, 6]).astype(int)

        holidays = calendar().holidays(start=X_['date'].min(), end=X_['date'].max())
        X_['is_holiday'] = X_['date'].isin(holidays).astype(int)       
        
        return X_.drop(columns=['date'])

In [29]:
add_temporal_features = TemporalFeaturesEngineer()
add_temporal_features.fit_transform(X_train)

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_2_hour,demand_previous_1_hour,sub_region_code,temperature_2m,average_demand_last_4_weeks,hour,day_of_week,month,is_weekend,is_holiday
0,11233.0,10916.0,10704.0,10387.0,10196.0,9905.0,9600.0,9254.0,9137.0,9045.0,...,8954.0,9607.0,53,0.108500,10073.50,2,6,1,1,0
1,11360.0,11147.0,11114.0,10904.0,10317.0,9762.0,9408.0,9056.0,8839.0,8796.0,...,8419.0,8419.0,53,0.158500,10384.25,1,0,1,0,0
2,12702.0,12591.0,12286.0,12122.0,11578.0,10916.0,10271.0,10111.0,9917.0,10061.0,...,9403.0,9696.0,53,1.158500,11313.25,0,1,1,0,0
3,11807.0,12549.0,12565.0,12310.0,12062.0,11694.0,11073.0,10446.0,10039.0,10055.0,...,9114.0,8829.0,53,6.358500,11268.50,0,2,1,0,0
4,10352.0,11299.0,12290.0,12391.0,12174.0,11998.0,11487.0,10970.0,10378.0,10148.0,...,11311.0,11422.0,53,2.408500,10960.75,23,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13404,4124.0,4124.0,4041.0,3928.0,3847.0,3624.0,3341.0,3116.0,2980.0,2893.0,...,5556.0,5698.0,71,23.358500,3997.00,21,2,6,0,0
13405,3843.0,3763.0,3763.0,3660.0,3585.0,3506.0,3360.0,3162.0,2965.0,2848.0,...,5272.0,5476.0,71,23.408501,4096.75,20,3,6,0,0
13406,4289.0,4395.0,4417.0,4417.0,4302.0,4122.0,3981.0,3758.0,3470.0,3263.0,...,4782.0,5080.0,71,21.158501,4282.50,19,4,6,0,0
13407,4280.0,4421.0,4537.0,4517.0,4517.0,4399.0,4217.0,4060.0,3811.0,3491.0,...,4671.0,4921.0,71,26.308500,4167.75,18,5,6,1,0


In [ ]:

from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_demand_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)
pipeline.fit(X_train, y_train);

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069331 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171978
[LightGBM] [Info] Number of data points in the train set: 13409, number of used features: 679
[LightGBM] [Info] Start training from score 3558.845999


In [31]:
import warnings
warnings.filterwarnings("ignore")

predictions = pipeline.predict(X_test)
print(' ')

In [32]:

from sklearn.metrics import mean_absolute_error
evaluate_model(y_test,predictions)

'MAE is 105.4326 and MAPE is: 0.4233'

In [33]:
from src.model_info import *

## lightBm with hyperprameter tunning

In [34]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

# from src.model_info import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100), 
        #"learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
  
    }
       
    tss = KFold(n_splits=6)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [35]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2025-01-27 16:46:33,053] A new study created in memory with name: no-name-7ccd8554-6292-41a3-8d20-e62ec4cc374b
[I 2025-01-27 16:47:30,997] Trial 0 finished with value: 90.28526399288589 and parameters: {'num_leaves': 205, 'feature_fraction': 0.5363472377925416, 'bagging_fraction': 0.45129625413624197, 'min_child_samples': 36}. Best is trial 0 with value: 90.28526399288589.
[I 2025-01-27 16:47:44,111] Trial 1 finished with value: 99.71483022341563 and parameters: {'num_leaves': 26, 'feature_fraction': 0.4105646898525159, 'bagging_fraction': 0.6715598203232624, 'min_child_samples': 39}. Best is trial 0 with value: 90.28526399288589.
[I 2025-01-27 16:48:53,234] Trial 2 finished with value: 102.98724195749821 and parameters: {'num_leaves': 237, 'feature_fraction': 0.8230252887482037, 'bagging_fraction': 0.2828703049981662, 'min_child_samples': 51}. Best is trial 0 with value: 90.28526399288589.
[I 2025-01-27 16:49:45,623] Trial 3 finished with value: 102.63972529455161 and parameters: {

In [36]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 205, 'feature_fraction': 0.5363472377925416, 'bagging_fraction': 0.45129625413624197, 'min_child_samples': 36}


In [37]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_demand_last_4_weeks at 0x000002BA0E5207C0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('standardscaler', StandardScaler()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.45129625413624197,
                               feature_fraction=0.5363472377925416,
                               min_child_samples=36, num_leaves=205))])

In [38]:
predictions = pipeline.predict(X_test)
evaluate_model(y_test, predictions)

'MAE is 102.7383 and MAPE is: 0.1875'